In [1]:

import math
def InitialDataReducedMass(orbitalangle,r0):
    phi=orbitalangle
    radiusseparation=r0 #separation one hundred
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi, radiusseparation,eccentricity, masses
    

In [7]:
def getxyuveqreducedmass(initdat): #using reduced mass cooordinate system
    phi,radiusseparation,eccentricity, masses=initdatreducedmass
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    x0=np.zeros(2)
    y0=np.zeros(2)
    x0[1]=radiusseparation*np.cos(phi)*metersperAU
    y0[1]=radiusseparation*np.sin(phi)*metersperAU
    if phi==0:
        print("zero")
        x0[1]=radiusseparation
        y0[1]=0
    if phi==math.pi:
        print("pi")
        x0[1]=-radiusseparation
        y0[1]=0
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        #rphys is r in the reduced mass system
        rphys[i]=radiusseparation #*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/radiusseparation
    x0phys[0]=0.0
    y0phys=rphys*y0/radiusseparation
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/(2*rphys)**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/(rphys))
    print(v)
    print("r0", r0)
    ux0=-v*y0/radiusseparation #[1]
    uy0=v*x0/radiusseparation #[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/((2*rphys)**2)
    ax0=-a*x0/radiusseparation
    ay0=-a*y0/radiusseparation
    az0=np.zeros(2)
    
    statevec=[]
    avec=[]
    for i in np.arange(len(x0phys)):
        stateveci=np.array([x0phys[i],y0phys[i],z0phys[i],ux0[i],uy0[i],uz0[i]])
        aveci=np.array([ax0[i],ay0[i],az0[i]])
        statevec.append(stateveci)
        avec.append(aveci)
    statevecnp=np.array(statevec)
    avecnp=np.array(avec)
    return masstotal, reducedmass,statevecnp,avecnp

In [8]:
import random,numpy as np
initdatreducedmass=InitialDataReducedMass(0,50)

In [11]:
xyuvaeqreducedmass=getxyuveqreducedmass(initdatreducedmass)
print(xyuvaeqreducedmass)
masstotal, reducedmass,xvec,avec=xyuvaeqreducedmass

zero
x0 [ 0. 50.]
y0 [0. 0.]
[1. 1.]
[2.  0.5]
[50. 50.]
[0.0004 0.0004]
[0.2 0.2]
r0 [ 0. 50.]
(2.0, array([2. , 0.5]), array([[ 0. ,  0. ,  0. , -0. ,  0. ,  0. ],
       [50. ,  0. ,  0. , -0. ,  0.2,  0. ]]), array([[-0.    , -0.    ,  0.    ],
       [-0.0002, -0.    ,  0.    ]]))


In [46]:
xvec[:,2]

array([0., 0.])

In [50]:
axii=np.zeros(len(xvec[:,0]))
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii

array([ 0., 50.])

In [12]:
def RK4implicit(h,t,xvec,f): #not a finite difference so no step in y
    k1= h*f(t,xvec)
    k2=h*f(t+h/2, xvec+k1/2)
    k3=h*f(t+h/2,xvec+k2/2)
    k4=h*f(t+h,xvec+k3)
    return t+h, xvec+1/6.*(k1+2.*k2+2.*k3+k4)

In [ ]:
class OrbitDiffEq:
    def __init__(self,reducedmass,xvec,avec,t0):
        self.reducedmass=reducedmass
        self.xvec=xvec
        self.ti=t0
        self.avec=avec
    def dxidt(self,t,xvec):
        return xvec[:,3].transpose()
    def dyidt(self,t,xvec):
        return xvec[:,4].transpose()
    def dzidt(self,t,xvec):
        return xvec[:,5].transpose()
    def dvxidt(self,t,xvec):
        #return axi[m]
        axii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    axii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    axii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,0]  - xvec[k,0])/rreljk**3
        return axii
    def dvyidt(self,t,xvec):
        #return axi[m]
        ayii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    ayii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    ayii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,1]  - xvec[k,1])/rreljk**3
        return ayii
    def dvzidt(self,t,xvec):
        #return axi[m]
        azii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    azii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    azii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,2]  - xvec[k,2])/rreljk**3
        return azii
    
    def dvecdt(self,t,xvec):
        avec=np.array([self.dvxidt(self,xvec),self.dvyidt(self,xvec),self.dvzidt(self,xvec)])
        avecT=avec.transpose()
        self.avec=avecT
        dvec2=np.array([self.dxidt(t,xvec),self.dyidt(t,xvec),self.dzidt(self,xvec),avec[0,:],avec[1,:],avec[2,:]])
                       
        dvec2T=dvec2.transpose()
        return dvec2T
    def updateINTERNAL(self,xvecii,tii):
        self.xvec=xvecii
        self.ti=tii
        return self
    def update(self,xvecii,avecii,tii):
        self.xvec=xvecii
        self.avec=avecii,
        self.ti=tii
    def print2D(self):
        print(self.reducedmass,self.xvec[1], self.ti)
        return self
    def list2D(self):
        return self.reducedmass,self.xvec,self.avec,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalxvec=RK4implicit(h,self.ti,self.xvec,self.dvecdt)
        #xii = intvalx
        #tnew,intvalvx=RK4implicit(h,self.ti,self.vxi,self.dvxidt)
        #vxii=intvalvx
        #tnew,intvaly=RK4implicit(h,self.ti,self.yi,self.dyidt)
        #yii = intvaly
        #tnew,intvalvy=RK4implicit(h,self.ti,self.vyi,self.dvyidt)
        #vyii=intvalvy
        #tnew,intvalz=RK4implicit(h,self.ti,self.zi,self.dzidt)
        #zii = intvalz
        #tnew,intvalvz=RK4implicit(h,self.ti,self.vzi,self.dvzidt)
        #vzii=intvalvz
        #axii=self.dvxidt(self.ti,np.zeros(np.shape(self.xi)))
        #ayii=self.dvyidt(self.ti,np.zeros(np.shape(self.yi)))
        #azii=self.dvzidt(self.ti,np.zeros(np.shape(self.zi)))
 
        #print(xii)
        self.updateINTERNAL(intvalxvec,tnew)
        return reducedmass, self.xvec,self.avec,self.ti

In [ ]:
mtotal,reducedmass,xvec,avec=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec,avec,0.0)
ODEeq.print2D()
ODEeq.print2D()

In [ ]:
dt=-.01 #natural units #*31556926 #seconds per year
numsteps=3000
mtotal, reducedmass,xvec0,avec0=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec0,avec0,0.0)
ODEeq.print2D()
t=0.0
star1x=[]
star2x=[]
star1a=[]
star2a=[]
for i in np.arange(1,numsteps):
    reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
    ODEeq.print2D()
    star1x.append(xvec[0])
    star2x.append(xvec[1])
    star1a.append(avec[0])
    star2a.append(avec[1])


